In [2]:
#Notebook 01 for Think Bayes
#Probability
import os
import pandas as pd
import pyreadstat
os.getcwd()

'/Users/matthewpahl/Documents/Learning/Math/Statistics/BayesianStats/think_bayes/01_Probability'

# Probability
Bayes theorem is the foundation of Bayesian statistics and Bayes theorem is based on conditional probability

### Example
Linda is 31, single, outspoken, and bright.  She majored in philosophy. She was deeply concerned with issues of social justice and discrimination. Which is more probabable? 
    1 Linda is a bank teller
    2 Linda is a bank teller AND is active in the feminist movement
    
Despite the information provided, 1 is more probable than 2, since the AND multiplies the probabilities together.

Defining probability is difficult, for our purposes
Probability: a fraction of a finite set

#toy data General Social Survey (GSS) 
Columns of interest
caseid - (index for table)
year - year of response
sex - male or female
polviews - policial views on a score from liberal to conservative
partyid - party affiliation, Democrat, Independent, or Republican
inds10 - code for industry the respondant was employed


In [3]:
#load 
cols = ["ID", "YEAR", "SEX", "POLVIEWS", "PARTYID", "INDUS10"]
gss = pd.read_spss('/Users/matthewpahl/Documents/Learning/Math/Statistics/BayesianStats/think_bayes/01_Probability/data/gss/GSS7218_R3.sav', usecols = cols)
gss.head()

,YEAR,ID,INDUS10,SEX,PARTYID,POLVIEWS
0,1972.0,1.0,Clothing stores,FEMALE,"IND,NEAR DEM",NaN
1,1972.0,2.0,Newspaper publishers,MALE,NOT STR DEMOCRAT,NaN
2,1972.0,3.0,Real estate,FEMALE,INDEPENDENT,NaN
3,1972.0,4.0,Clothing stores,FEMALE,NOT STR DEMOCRAT,NaN
4,1972.0,5.0,Wired telecommunications carriers,FEMALE,STRONG DEMOCRAT,NaN


#check number of real estate agents in dataset (bankers were coded weird)

In [4]:
realEstate = (gss['INDUS10'] == 'Real estate')
realEstate.head()

0    False
1    False
2     True
3    False
4    False
Name: INDUS10, dtype: bool

In [5]:
realEstate.sum()

913

In [6]:
def prob(A):
    """Compute probability of proposition, A."""
    return A.mean()

prob(realEstate)

0.014086462801246645

~1.4% of the respondants are real estate agents, so randomly selecting particiapants has a 1.4% probability of being a real estate agent. Next can check the sex of participants (true = female, false = male in the following)

In [7]:
female = gss['SEX'] == 'FEMALE'
prob(female)

0.5585213071250038

More women than men in the dataset.  This is because GSS does not include people in the military or prison which are more likely to be men.

In [8]:
lib_cat = ['SLIGHTLY LIBERAL', 'LIBERAL', 'EXTREMELY LIBERAL']
lib = gss['POLVIEWS'].isin(lib_cat)
prob(lib)

0.234609806523282

#Conjunction
Name for the logic "AND" operation

A AND B are true, if and only if A is true, and B is true. Otherwise the conjugated statement is FALSE
This relationship is communitative (B AND A is the same as A AND B)


In [9]:
#Conjuction
prob(realEstate)

0.014086462801246645

In [10]:
prob(realEstate & lib)

0.0033943283858425648

Probability of liberal real estate agents is lower than just real estate agents. This is expected as not all real estate agents are liberal

#Conditional Probability
the probability depending on a 
Ex
What is the probability that a respondent is female, given they are a real estate agent

In [11]:
#fraction of liberals that are female
selected = female[lib]
prob(selected)

0.5542548993818229

In [12]:
#define conditional to compute probability given proposition and given
def conditional(prop, given):
    """Probability of prop conditioned on given."""
    return(prob(prop[given]))

In [13]:
conditional(lib, given=female)

0.23281767955801105

Conditional probability is not communicative
P(A|B) is not the same as P(B|A)

In [14]:
conditional(female, realEstate)

0.5509309967141293

In [15]:
conditional(realEstate, female)

0.013895027624309391

So while 55.1% of real estate agents are female, only 1.4% of female respondants are real estate agents.

Condition and Conjunction
Can combine conditional probability and conjunctions


In [16]:
conditional(realEstate & lib, female)

0.0033701657458563537

In [17]:
conditional(realEstate, female & lib)

0.014475557664926435

about 0.3% of female respondants are liberal real estate agents, and about 1.4% of real estate agents are liberal female respondants

# Laws of probability
### Theorem 1: P(A|B) = P(A & B) / P(A)

In [20]:
prob(female & realEstate) / prob(realEstate)

0.5509309967141293

In [19]:
conditional(female, given=realEstate)

0.5509309967141293

### Theorem 2: P(A & B) = P(A|B) * P(B)
Just algebraic rearrangement of theorem 1
Probability of A and B equals the probability of A given B multiplied by probability of B. Implies that rather than using the AND operator, can just multiple two probabilities.

In [21]:
prob(female & realEstate)

0.007760668991267319

In [22]:
prob(female) * conditional(realEstate, given = female)

0.0077606689912673174

### Theorem 3: 
Conjuction is Commutative:
    P(A and B) = P(B and A) (1)
Apply theorem 2 to both sides
    P(A|B) * P(B) = P(B|A) * P(A)
Divide by P(B) on both sides:
    P(A|B) = P(B|A) * P(A)/P(B) which is Bayes Theorem

In [23]:
conditional(lib, given = realEstate)

0.24096385542168675

In [24]:
conditional(realEstate, given = lib) * prob(lib) / prob(realEstate)

0.24096385542168677

# Law of total probability
P(A) = P(A|B1) + P(A|B2), where B1 and B2 are mutually exclusive and exhaustive

Applying theorem 2.
P(A) = P(B1)*P(A|B1) + P(B2)*P(A|B2)

Which can be generalized to

P(A) = SUM( P(Bi) * P(A|Bi) ) 
With the conditions that Bi are mutually exclusive and exhaustive